# Apache Spark. Spark ML.

In [ ]:
import os
JAVA_HOME = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["JAVA_HOME"]= JAVA_HOME

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder \
    .appName("my_spark") \
    .master("local[*]") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .getOrCreate()

In [ ]:
spark

In [ ]:
import pyspark.sql.functions as f
from pyspark.sql.types import *

## Ключевые понятия

* **Pipeline**  
Организация серии трансформеров и эстиматоров в единый конвейер

* **Estimator**
Алгоритм непосредственно обучения, возвращает модель, которая является трансформером

![](https://spark.apache.org/docs/latest/img/ml-Pipeline.png)

* **Transformer**  
Алгоритм, который трансформирует один датафрейм в другой на основе заданных правил. Это касается как предсказаний, так и фиче-инжиниринга

![](https://spark.apache.org/docs/latest/img/ml-PipelineModel.png)

In [ ]:
from pyspark.ml.linalg import Vectors

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
train = spark.createDataFrame([
    (1.0, Vectors.dense([0.0, 1.1, 0.1])),
    (0.0, Vectors.dense([2.0, 1.0, -1.0])),
    (0.0, Vectors.dense([2.0, 1.3, 1.0])),
    (1.0, Vectors.dense([0.0, 1.2, -0.5]))], schema = ["label", "features"])

In [ ]:
train.show()

In [ ]:
LogisticRegression?

In [ ]:
lr = LogisticRegression(maxIter=10, regParam=0.01)

In [ ]:
from pyspark.ml import Estimator, Transformer

In [ ]:
isinstance(lr, Estimator)

In [ ]:
type(lr)

In [ ]:
lr.params

In [ ]:
lr.getOrDefault("regParam")

In [ ]:
train.rdd.getNumPartitions()

In [ ]:
train.repartition(10).rdd.getNumPartitions()

In [ ]:
model = lr.fit(train)

In [ ]:
type(model)

In [ ]:
isinstance(model, Transformer)

In [ ]:
model.coefficients

In [ ]:
model.interceptVector

In [ ]:
predict = model.transform(train)

In [ ]:
predict.printSchema()

In [ ]:
predict.show(10, truncate=False, vertical=True)

In [ ]:
model.getOrDefault("threshold")

## Токсичные комментарии!

Давайте изучим этот замечтальный датасет!

Скачать его можно тут:
https://www.kaggle.com/competitions/jigsaw-toxic-comment-classification-challenge

Давайте загрузим его в DataFrame. А как?

In [ ]:
!head toxic/train.csv

schema = ...

In [ ]:
schema = StructType([
    StructField("id", StringType()),
    StructField("comment_text", StringType()),
    StructField("toxic", IntegerType()),
    StructField("severe_toxic", IntegerType()),
    StructField("obscene", IntegerType()),
    StructField("threat", IntegerType()),
    StructField("insult", IntegerType()),
    StructField("identity_hate", IntegerType())
])

dataset = spark.read.csv...

In [ ]:
spark.read.csv?

In [ ]:
dataset = spark.read.csv("toxic/train.csv", schema=schema, header=True)
dataset.show(5)

In [ ]:
dataset = spark.read.csv("toxic/train.csv", schema=schema, header=True, multiLine=True)

Некрасиво смотрится, давайте добавим форматирования!

In [ ]:
dataset.show(5, False, True)

Ой, кажется, что-то пошло не так. Может, добавим какие-то опции?

In [ ]:
!head toxic/train.csv

Давайте посмотрим опции в spark.read.csv?

In [ ]:
spark.read.csv?

In [ ]:
spark.read.csv("toxic/train.csv", schema=schema, 
                         header=True, multiLine=True).show()

In [ ]:
dataset = spark.read.csv("toxic/train.csv", schema=schema, 
                         header=True, multiLine=True, escape='"')

In [ ]:
dataset.show(5, False, True)

In [ ]:
dataset.select("id").show(100)

In [ ]:
dataset.show(2, vertical=True, truncate=False)

In [ ]:
dataset.rdd.getNumPartitions()

In [ ]:
dataset.count()

In [ ]:
dataset = dataset.repartition(4).cache()

In [ ]:
dataset.count()

### Начнеем с бинарной классификации (toxic/non-toxic)

Если вы пропустили импорты, то не забудьте про  
from pyspark.sql import functions as f

Давайте превратим большую шкалу в бинарную через функцию when

In [ ]:
f.when?

In [ ]:
dataset.printSchema()

In [ ]:
target = f.when(
    (dataset.toxic == 0) &
    (dataset.severe_toxic == 0) &
    (dataset.obscene == 0) &
    (dataset.threat == 0) &
    (dataset.insult == 0) &
    (dataset.identity_hate == 0),
    0
).otherwise(1)

In [ ]:
target_other = f.when(
    (f.col("toxic") == 0) &
    (f.col("severe_toxic") == 0) &
    (f.col("obscene") == 0) &
    (f.col("threat") == 0) &
    (f.col("insult") == 0) &
    (f.col("identity_hate") == 0),
    0
).otherwise(1)

In [ ]:
target

In [ ]:
target_other

In [ ]:
my_dataset = spark.read.csv("toxic/train.csv", schema=schema, 
                         header=True, multiLine=True, escape='"')

In [ ]:
my_dataset.withColumn("target", target_other).show()

In [ ]:
dataset = dataset.withColumn("target", target)

In [ ]:
dataset.select("id", "target").show(10)

In [ ]:
targets = dict(dataset.groupBy("target").count().collect())

In [ ]:
targets

In [ ]:
dataset.filter(f.col("target") == 1).select("comment_text").show(5,False,True)

In [ ]:
targets[1] / (targets[0] + targets[1])

In [ ]:
dataset = dataset.drop("toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate").cache()

In [ ]:
dataset

In [ ]:
dataset.show(2, False, True)

In [ ]:
dataset.write.parquet("toxic_dataset", mode="overwrite")

In [ ]:
dataset.printSchema()

In [ ]:
dataset = spark.read.parquet("toxic_dataset")

###  Построим простую бинарную логистическую регрессию

In [ ]:
from pyspark.ml.feature import *

### Займемся токенами!

In [ ]:
Tokenizer?

Нам надо создать токенайзер и натравить его на поле с комментариями, создать с его помощью новый датасет. Если забыли, то вам нужна операция transform

In [ ]:
tokenizer = Tokenizer(inputCol="comment_text", outputCol="words")

In [ ]:
dataset2 = tokenizer.transform(dataset)

In [ ]:
dataset2.select("id", "words").show(2, False, True)

### Конвертируем наши слова в вектора с помощью hashing trick

А кстати, что это?

In [ ]:
HashingTF?

Допустим, количество фичей у нас 100. Классификация, напомним, у нас бинарная.

In [ ]:
hasher = HashingTF(numFeatures=100, binary=True, inputCol=tokenizer.getOutputCol(), outputCol="word_vector")

In [ ]:
dataset2 = hasher.transform(dataset2)

In [ ]:
dataset2.select("id", "word_vector").show(2, False, True)

### Настала пора разбить на тест и трейн!
Классы у нас несбалансированные, поэтому надо подкрутить параметры

In [ ]:
dataset2.sampleBy?

In [ ]:
train = dataset2.sampleBy("target", fractions={0: 0.8, 1: 0.8}, seed=42)

In [ ]:
train_targets = dict(train.groupby("target").count().collect())

In [ ]:
train_targets

In [ ]:
train_targets[1] / (train_targets[0] + train_targets[1])

А как можно с помощью join взять оставшиеся данные?

In [ ]:
dataset2.join?

In [ ]:
test = dataset2.join(train, on="id", how="leftanti")

In [ ]:
test_targets = dict(test.groupby("target").count().collect())

Сохранилось ли распределение?

In [ ]:
test_targets[1] / (test_targets[0] + test_targets[1])

In [ ]:
train.rdd.getNumPartitions()

In [ ]:
test.rdd.getNumPartitions()

In [ ]:
train = train.drop("comment_text", "words").cache()

In [ ]:
test = test.drop("comment_text", "words").coalesce(4).cache()

In [ ]:
test.rdd.getNumPartitions()

### Настало время для фита!

Начем с логистической регрессии. Ограничимся 15 итерациями, чтобы не усложнять себе жизнь.

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
LogisticRegression?

In [ ]:
hasher.getOutputCol()

In [ ]:
lr = LogisticRegression(featuresCol=hasher.getOutputCol(), labelCol="target", maxIter=15)

Зафитим!

In [ ]:
lr_model = lr.fit(train)

In [ ]:
lr_model?

Запредиктим!

In [ ]:
predictions = lr_model.transform(test)

In [ ]:
predictions?

In [ ]:
predictions.printSchema()

In [ ]:
predictions.select("id", "target", "prediction", "probability", "rawPrediction").show(10, False, True)

А если хотим Accuracy посчитать?

In [ ]:
true_predictions = predictions.select("target", f.col("prediction").cast("int")).filter("target == prediction").count()

In [ ]:
true_predictions

In [ ]:
print("Accuracy is {}".format(true_predictions / predictions.count()))

### Это хорошо? Может быть...

In [ ]:
predictions.select("target", f.col("prediction").cast("int"))\
           .filter((f.col("target") == 1) & (f.col("prediction") == f.col("target")))\
           .count()

In [ ]:
predictions.printSchema()

In [ ]:
predictions_pd = predictions.select("target", f.col("prediction").cast("int")).toPandas()

In [ ]:
predictions_pd.head()

In [ ]:
lr.getOrDefault("threshold")

In [ ]:
!pip install scikit-learn
from sklearn.metrics import classification_report, precision_score

In [ ]:
print(classification_report(predictions_pd.target, predictions_pd.prediction))

In [ ]:
train.count()

### А если хочется что-то посложнее?

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
BinaryClassificationEvaluator?

In [ ]:
evaluator = BinaryClassificationEvaluator(labelCol="target", metricName='areaUnderROC')

In [ ]:
evaluator.evaluate(predictions)

In [ ]:
evaluator.setParams(metricName="precision")

In [ ]:
evaluator.evaluate(predictions)

### Упс :(

`spark.ml.evaluation.BinaryClassificationEvaluator` supports only ROC AUC and PR AUC.

### Так а если хотим больше?
Давайте немного поколхозим!

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
MulticlassClassificationEvaluator?

In [ ]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="target", 
                                              metricName="accuracy")

In [ ]:
evaluator.evaluate(predictions)

А если weightedPrecision или weightedRecall?

In [ ]:
evaluator?

In [ ]:
evaluator = evaluator.setMetricName("weightedPrecision")

In [ ]:
evaluator.evaluate(predictions)

In [ ]:
evaluator = evaluator.setMetricName("weightedRecall")

In [ ]:
evaluator.evaluate(predictions)

### А теперь все вместе!

In [ ]:
dataset = spark.read.parquet("toxic_dataset")

In [ ]:
!ls toxic_dataset

In [ ]:
dataset

In [ ]:
dataset.rdd.getNumPartitions()

In [ ]:
from pyspark.ml import Pipeline

Соберем пайплайн!

In [ ]:
Pipeline?

In [ ]:
pipeline = Pipeline(stages=[
    tokenizer,
    hasher,
    lr
])

Разбиваем на трейн и тест. Снова :)

In [ ]:
train = dataset.sampleBy("target", fractions={0: 0.8, 1: 0.8}).cache()

In [ ]:
test = dataset.join(train, on="id", how="leftanti").cache()

Фит и предикт. Снова :)

In [ ]:
pipeline_model = pipeline.fit(train)

In [ ]:
pipeline_model

In [ ]:
predictions = pipeline_model.transform(test)

Эвалуатор на ваш вкус

In [ ]:
evaluator = BinaryClassificationEvaluator(labelCol="target", metricName='areaUnderROC')

In [ ]:
evaluator.evaluate(predictions)

### Так, а есть что-то позаковыристее?

In [ ]:
from pyspark.ml.classification import GBTClassifier

In [ ]:
GBTClassifier?

In [ ]:
gbt = GBTClassifier(featuresCol=hasher.getOutputCol(), labelCol="target", maxIter=20, maxDepth=3)

Давайте добавим в пайплайн!

In [ ]:
pipeline = Pipeline(stages=[
    tokenizer,
    hasher,
    gbt
])

Фит, предикт, проверка

In [ ]:
pipeline_model = pipeline.fit(train)

In [ ]:
predictions = pipeline_model.transform(test)

In [ ]:
evaluator.evaluate(predictions)

### Добавим степеней свободы
Допустим, теперь мы хотим увеличить количество фичей в хэшировании

In [ ]:
pipeline_model.stages

In [ ]:
pipeline.fit?

In [ ]:
hasher?

In [ ]:
pipeline_model = pipeline.fit(train, params={hasher.numFeatures: 1000})

In [ ]:
pipeline_model.stages[1].extractParamMap()

In [ ]:
predictions = pipeline_model.transform(test)

In [ ]:
evaluator.evaluate(predictions)

### Почистим от стопслов
Давайте почистим датасет от мусорных слов!

In [ ]:
StopWordsRemover?

In [ ]:
stop_words = StopWordsRemover.loadDefaultStopWords("english")

In [ ]:
stop_words

In [ ]:
swr = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol="words_filtered", 
                       stopWords=stop_words)

In [ ]:
hasher = HashingTF(numFeatures=1000, binary=True, inputCol=swr.getOutputCol(), 
                   outputCol="word_vector")

А куда мы его в пайплане должны поместить?

In [ ]:
pipeline = Pipeline(stages=[
    tokenizer,
    swr,
    hasher,
    lr
])

In [ ]:
pipeline_model = pipeline.fit(train)

In [ ]:
pipeline_model.stages

In [ ]:
predictions = pipeline_model.transform(test)

In [ ]:
evaluator.evaluate(predictions)

### Даешь еще фичей!

In [ ]:
dataset.printSchema()

А давайте добавим длину комментария?

In [ ]:
dataset = dataset.withColumn("comment_length", f.length(dataset.comment_text))

In [ ]:
dataset.show()

In [ ]:
train = dataset.sampleBy("target", fractions={0: 0.8, 1: 0.8}).cache()
test = dataset.join(train, on="id", how="leftanti").cache()

### Не забываем, что фичи должны быть в одной колонке. `VectorAssembler` нам поможем!

In [ ]:
VectorAssembler?

In [ ]:
assembler = VectorAssembler(inputCols=[hasher.getOutputCol(), "comment_length"], outputCol="features")

In [ ]:
lr = LogisticRegression(labelCol="target", maxIter=15)

In [ ]:
pipeline = Pipeline(stages=[
    tokenizer,
    swr,
    hasher,
    assembler,
    lr
])

In [ ]:
pipeline_model = pipeline.fit(train)

In [ ]:
pipeline_model.stages

In [ ]:
predictions = pipeline_model.transform(test)

In [ ]:
evaluator.evaluate(predictions)

In [ ]:
pipeline_model.stages[-1].coefficients[-1]

### А что там вообще другие придумали? 
https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/discussion/52557

In [ ]:
CountVectorizer?

In [ ]:
count_vectorizer = CountVectorizer(inputCol=swr.getOutputCol(), outputCol="word_vector", binary=True)

In [ ]:
assembler = VectorAssembler(inputCols=[count_vectorizer.getOutputCol(), "comment_length"], outputCol="features")

In [ ]:
pipeline = Pipeline(stages=[
    tokenizer,
    swr,
    count_vectorizer,
    assembler,
    lr
])

In [ ]:
pipeline_model = pipeline.fit(train)

In [ ]:
predictions = pipeline_model.transform(test)

In [ ]:
evaluator.evaluate(predictions)

### Что ж это за ML без тюнинга гиперпараметров?

In [ ]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [ ]:
CrossValidator?

In [ ]:
ParamGridBuilder?

In [ ]:
paramGrid = ParamGridBuilder().addGrid(count_vectorizer.vocabSize, [100, 500])\
                              .addGrid(lr.regParam, [0.01, 0.05])\
                              .build()

In [ ]:
paramGrid

In [ ]:
crossval = CrossValidator(estimator=pipeline, estimatorParamMaps=paramGrid,
                          evaluator=evaluator, numFolds=3, parallelism=4)

In [ ]:
cv_model = crossval.fit(train)

In [ ]:
cv_model.avgMetrics

In [ ]:
cv_model.bestModel

In [ ]:
predictions = cv_model.transform(test)

In [ ]:
evaluator.evaluate(predictions)

### А если я хочу больше разных моделей?

#### 1. Обучим случайный лес локально

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [ ]:
X, y = make_classification(random_state=42)

In [ ]:
y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape

In [ ]:
est = LogisticRegression(random_state=42)

In [ ]:
est.fit(X_train, y_train)

#### 2. Теперь уложим это в Spark

In [ ]:
from pyspark.ml.linalg import DenseVector
from pyspark.ml.linalg import VectorUDT

Spark требует немного другой формат. Конвертируем!

In [ ]:
schema = StructType(fields=[
    StructField("features", VectorUDT()),
    StructField("label", IntegerType())
])

In [ ]:
df_test = spark.createDataFrame(zip(map(DenseVector, X_test), map(int, y_test)), schema=schema)

In [ ]:
df_test.show()

Добавим pandas_udf

In [ ]:
est_broadcast = spark.sparkContext.broadcast(est)

In [ ]:
import pandas as pd

In [ ]:
@f.pandas_udf(FloatType())
def predict(features):
    predictions = est_broadcast.value.predict(features)
    return pd.Series(predictions)

In [ ]:
df_test.withColumn("prediction", predict("features")).show()

Ой, в PyArrow нет векторов :(

In [ ]:
from pyspark.ml.functions import vector_to_array

In [ ]:
df_test.withColumn("prediction", predict(vector_to_array("features"))).show()

И еще немного напильника...

In [ ]:
@f.pandas_udf(FloatType())
def predict(series):
    predictions = est_broadcast.value.predict(series.tolist())
    return pd.Series(predictions)

In [ ]:
df_test.withColumn("prediction", predict(vector_to_array("features"))).show()

In [ ]:
spark.stop()